In [1]:
import torchreid

torchreid.models.show_avai_models()


c:\Users\merth\AppData\Local\Programs\Python\Python39\lib\site-packages\torchreid\reid\metrics\rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(


['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'resnext50_32x4d', 'resnext101_32x8d', 'resnet50_fc512', 'se_resnet50', 'se_resnet50_fc512', 'se_resnet101', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'densenet121_fc512', 'inceptionresnetv2', 'inceptionv4', 'xception', 'resnet50_ibn_a', 'resnet50_ibn_b', 'nasnsetmobile', 'mobilenetv2_x1_0', 'mobilenetv2_x1_4', 'shufflenet', 'squeezenet1_0', 'squeezenet1_0_fc512', 'squeezenet1_1', 'shufflenet_v2_x0_5', 'shufflenet_v2_x1_0', 'shufflenet_v2_x1_5', 'shufflenet_v2_x2_0', 'mudeep', 'resnet50mid', 'hacnn', 'pcb_p6', 'pcb_p4', 'mlfn', 'osnet_x1_0', 'osnet_x0_75', 'osnet_x0_5', 'osnet_x0_25', 'osnet_ibn_x1_0', 'osnet_ain_x1_0', 'osnet_ain_x0_75', 'osnet_ain_x0_5', 'osnet_ain_x0_25']


In [2]:
model = torchreid.models.build_model(
    name='osnet_x0_25',  # hafif versiyon
    num_classes=1000,
    pretrained=True
)

model.eval()


Successfully loaded imagenet pretrained weights from "C:\Users\merth/.cache\torch\checkpoints\osnet_x0_25_imagenet.pth"


OSNet(
  (conv1): ConvLayer(
    (conv): Conv2d(3, 16, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv2): Sequential(
    (0): OSBlock(
      (conv1): Conv1x1(
        (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
      (conv2a): LightConv3x3(
        (conv1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
        (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
      (conv2b): Sequential(
        (

In [3]:
import torch
dummy_input = torch.randn(1, 3, 256, 128)
torch.onnx.export(model, dummy_input, "reid_model.onnx", opset_version=11)


In [7]:
import tensorflow as tf

class ReIDModel(tf.Module):
    @tf.function(input_signature=[tf.TensorSpec([None, 256, 128, 3], tf.float32)])
    def __call__(self, x):
        # Bu modelde "reid_model_tf" klasöründen yüklenen gerçek model kullanılmalı
        return self.model(x)

    def __init__(self):
        self.model = tf.saved_model.load("reid_model_tf")  # yüklenmiş TensorFlow modeli


In [8]:
model = ReIDModel()
tf.saved_model.save(model, "reid_model_tf_fixed", signatures={"serving_default": model.__call__.get_concrete_function()})


INFO:tensorflow:Assets written to: reid_model_tf_fixed\assets


In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("reid_model_tf_fixed")
tflite_model = converter.convert()

with open("reid_model.tflite", "wb") as f:
    f.write(tflite_model)
